In [350]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
x = 2021

url = f"https://fbref.com/en/comps/19/{x}-{x+1}/schedule/{x}-{x+1}-Europa-League-Scores-and-Fixtures"
# url = f'https://fbref.com/en/comps/8/{x}-{x+1}/schedule/{x}-{x+1}-Champions-League-Scores-and-Fixtures'
# url = f'https://fbref.com/en/comps/882/{x}-{x+1}/schedule/{x}-{x+1}-Conference-League-Scores-and-Fixtures'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")

In [352]:
table = soup.find("table", {"class": "stats_table"})

header_row = table.find("thead").find("tr") if table.find("thead") else table.find("tr")
headers = [th.text.strip() for th in header_row.find_all("th")]

# Extract rows
rows = []
for tr in table.find("tbody").find_all("tr"):  # Use tbody to avoid header rows
    cells = [td.text.strip() for td in tr.find_all("td")]
    if cells:  # Ignore empty rows
        rows.append(cells)

# Ensure column and row lengths match
if len(headers) > len(rows[0]):  # If there are extra headers
    headers = headers[-len(rows[0]):]  # Use only the last N headers matching the data

# Convert to DataFrame
df = pd.DataFrame(rows, columns=headers)

df[['Home', 'Home_Country']] = df['Home'].str.rsplit(n=1, expand=True)
df[['Away_Country', 'Away']] = df['Away'].str.split(n=1, expand=True)

df = df[['Date', 'Home', 'Away', 'Home_Country', 'Away_Country', 'Score']]

df.columns = ['date', 'hometeam', 'awayteam', 'homecountry', 'awaycountry', 'score']

df['score'] = df['score'].str.replace(r'\(.*?\)\s*', '', regex=True)

df[['homescore', 'awayscore']] = df['score'].str.split('–', expand=True)

# Convert the score columns to numeric
df['homescore'] = pd.to_numeric(df['homescore'])
df['awayscore'] = pd.to_numeric(df['awayscore'])

# Create a new column for match result
df['result'] = df.apply(
    lambda row: 0 if row['homescore'] > row['awayscore'] else (0.5 if row['homescore'] == row['awayscore'] else 1),
    axis=1
)

df.drop(columns=['score'], inplace=True)

df.dropna(inplace=True)

In [ ]:
df.to_csv(f'europe/europa_league/{x}-{x+1}.csv')